In [1]:
import openclean.data.masterdata as masterdata

masterdata.download('restcountries', 'countries', replace=True)
# masterdata.download('d4')

In [2]:
countries = masterdata.load('restcountries', 'countries')
countries.head()

,name,alpha2Code,alpha3Code,capital,region,subregion
1,Afghanistan,AF,AFG,Kabul,Asia,Southern Asia
4,Åland Islands,AX,ALA,Mariehamn,Europe,Northern Europe
5,Albania,AL,ALB,Tirana,Europe,Southern Europe
64,Algeria,DZ,DZA,Algiers,Africa,Northern Africa
10,American Samoa,AS,ASM,Pago Pago,Oceania,Polynesia


In [3]:
countries.loc[countries['name'] == 'Venezuela (Bolivarian Republic of)']

,name,alpha2Code,alpha3Code,capital,region,subregion
239,Venezuela (Bolivarian Republic of),VE,VEN,Caracas,Americas,South America


In [4]:
from openclean.operator.transform.update import update

countries = update(countries, 'name', {'Venezuela (Bolivarian Republic of)': 'Venezuela'})

In [5]:
countries.loc[countries['name'] == 'Venezuela']

,name,alpha2Code,alpha3Code,capital,region,subregion
239,Venezuela,VE,VEN,Caracas,Americas,South America


In [6]:
countries.head()

,name,alpha2Code,alpha3Code,capital,region,subregion
1,Afghanistan,AF,AFG,Kabul,Asia,Southern Asia
4,Åland Islands,AX,ALA,Mariehamn,Europe,Northern Europe
5,Albania,AL,ALB,Tirana,Europe,Southern Europe
64,Algeria,DZ,DZA,Algiers,Africa,Northern Africa
10,American Samoa,AS,ASM,Pago Pago,Oceania,Polynesia


In [7]:
masterdata.update('restcountries', 'countries', countries)

In [8]:
countries = masterdata.load('restcountries', 'countries', version=1)
countries.loc[countries['name'] == 'Venezuela']

,name,alpha2Code,alpha3Code,capital,region,subregion
239,Venezuela,VE,VEN,Caracas,Americas,South America


In [9]:
snapshots = masterdata.snapshots('restcountries', 'countries')
for s in snapshots:
    print(s)

<Snapshot (version=0 description='' at=2020-05-19 15:54:07.478064-04:00)>
<Snapshot (version=1 description='' at=2020-05-19 15:54:07.740146-04:00)>


In [10]:
prov = masterdata.diff('restcountries', 'countries', 0, 1)
prov.describe()

Schema Changes
Inserted Columns : 0
Deleted Columns  : 0
Moved Columns    : 0
Renamed Columns  : 0

Data Changes
Inserted Rows    : 0
Deleted Rows     : 0
Moved Rows       : 0
Updated Rows     : 1
Updated Values   : 1


In [11]:
row = prov.rows().update()[0]
old_val, new_val = row.cells[0].values()

print("Country name updated from '{}' to '{}'".format(old_val, new_val))

Country name updated from 'Venezuela (Bolivarian Republic of)' to 'Venezuela'
